In [14]:
import pandas as pd
import pickle
from collections import Counter
import copy

import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier




In [2]:

# pd.set_option('display.max_rows', 300)

# survey = pd.read_csv('/Users/williamyee/Documents/Data Bootcamp/Classification-Project-DSjobs/Data/kaggle-survey-2017/multi_r_.csv', encoding = 'ISO-8859-1')

# list(survey.columns)

# survey.JobSatisfaction.value_counts()



# survey.head()

# survey.EmploymentStatus.unique()

# survey.groupby('EmploymentStatus').count()

# # survey.UniversityImportance.value_counts()

# # survey.JobSatisfaction.value_counts()




In [3]:
# conditions = [  survey.JobSatisfaction.isin(['7','8','9','10 - Highly Satisfied']),
#                 survey.JobSatisfaction.isin(['1 - Highly Dissatisfied','2','3','4','5','6','I prefer not to share'])
#              ]
# choices = [1,0]
# survey['JobSatisfaction'] = np.select(conditions, choices, default=survey.JobSatisfaction)


# survey = survey[survey.JobSatisfaction.notna()]

In [4]:
# percent_missing = survey.isnull().sum()*100/len(survey)
# missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
# missing_value_df

In [5]:
# percent_missing = survey.isnull().sum()*100/len(survey)
# missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
# seventy_five_data = missing_value_df[missing_value_df.percent_missing<25]

# seventy_five_data

# survey_minimal_missing= survey[list(seventy_five_data.columnname)]

# survey_minimal_missing.reset_index(inplace=True)
# X = survey_minimal_missing.drop(columns = 'JobSatisfaction',axis =1)
# y = survey_minimal_missing['JobSatisfaction']
# # X

In [6]:
# X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=100)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2, random_state=100)

In [7]:
# X_cat_train = X_train.select_dtypes(include='object')
# X_cat_train = pd.DataFrame(X_cat_train,dtype ='str')
# imp_freq_train = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# X_cat_imputed_train = imp_freq_train.fit_transform(X_cat_train)
# X_cat_imputed_train = pd.DataFrame(data = X_cat_imputed_train)

# OE = OneHotEncoder(handle_unknown='ignore')
# OE.fit(X_cat_imputed_train)
# X_cat_imputed_train = OE.transform(X_cat_imputed_train)
# names= OE.get_feature_names()[:]
# names = names.tolist()
# X_cat_imputed_train = pd.DataFrame(data = X_cat_imputed_train.toarray(), columns = names)




# X_num_train = X_train.select_dtypes(include='float64')
# imp_freq_train = SimpleImputer(missing_values=np.nan, strategy='median')
# X_num_imputed_train = imp_freq_train.fit_transform(X_num_train)
# X_num_imputed_train = pd.DataFrame(data= X_num_imputed_train, columns = X_num_train.columns)

# scale = StandardScaler()
# scale.fit(X_num_imputed_train)
# X_num_imputed_train = scale.transform(X_num_imputed_train)
# X_num_imputed_train = pd.DataFrame(data= X_num_imputed_train, columns = X_num_train.columns)


# X_train_processed = pd.concat([X_cat_imputed_train,X_num_imputed_train],axis =1)

# y_train =pd.Series(y_train, dtype ='int')

In [8]:
# X_cat_test = X_test.select_dtypes(include='object')
# X_cat_test = pd.DataFrame(X_cat_test,dtype ='str')
# imp_freq_test = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# X_cat_imputed_test = imp_freq_test.fit_transform(X_cat_test)
# X_cat_imputed_test = pd.DataFrame(data = X_cat_imputed_test)

# X_cat_imputed_test = OE.transform(X_cat_imputed_test)
# names= OE.get_feature_names()[:]
# names = names.tolist()
# X_cat_imputed_test = pd.DataFrame(data = X_cat_imputed_test.toarray(), columns = names)




# X_num_test = X_test.select_dtypes(include='float64')
# imp_freq_test = SimpleImputer(missing_values=np.nan, strategy='median')
# X_num_imputed_test = imp_freq_test.fit_transform(X_num_test)
# X_num_imputed_test = pd.DataFrame(data= X_num_imputed_test, columns = X_num_test.columns)

# scale = StandardScaler()
# scale.fit(X_num_imputed_test)
# X_num_imputed_test = scale.transform(X_num_imputed_test)
# X_num_imputed_test = pd.DataFrame(data= X_num_imputed_test, columns = X_num_test.columns)


# X_test_processed = pd.concat([X_cat_imputed_test,X_num_imputed_test],axis =1)

# y_test =pd.Series(y_test, dtype ='int')

In [9]:
# X_cat_val = X_val.select_dtypes(include='object')
# X_cat_val = pd.DataFrame(X_cat_val,dtype ='str')
# imp_freq_val = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# X_cat_imputed_val = imp_freq_val.fit_transform(X_cat_val)
# X_cat_imputed_val = pd.DataFrame(data = X_cat_imputed_val)


# X_cat_imputed_val = OE.transform(X_cat_imputed_val)
# names= OE.get_feature_names()[:]
# names = names.tolist()
# X_cat_imputed_val = pd.DataFrame(data = X_cat_imputed_val.toarray(), columns = names)




# X_num_val = X_val.select_dtypes(include='float64')
# imp_freq_val = SimpleImputer(missing_values=np.nan, strategy='median')
# X_num_imputed_val = imp_freq_val.fit_transform(X_num_val)
# X_num_imputed_val = pd.DataFrame(data= X_num_imputed_val, columns = X_num_val.columns)

# scale = StandardScaler()
# scale.fit(X_num_imputed_val)
# X_num_imputed_val = scale.transform(X_num_imputed_val)
# X_num_imputed_val = pd.DataFrame(data= X_num_imputed_val, columns = X_num_val.columns)


# X_val_processed = pd.concat([X_cat_imputed_val,X_num_imputed_val],axis =1)

# y_val =pd.Series(y_val, dtype ='int')

In [10]:
# X_imputed.sample(20)

In [11]:
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.over_sampling import SMOTE, ADASYN


# ros = RandomOverSampler()
# X_train_os, y_train_os = ros.fit_sample(X_train_processed, y_train)


# smot = SMOTE()
# X_train_smote, y_train_smote = smot.fit_sample(X_train_processed,y_train)

# adasyn = ADASYN()
# X_train_ada, y_train_ada = adasyn.fit_sample(X_train_processed,y_train)



In [12]:
# pickle.dump(X_train_processed,open('X_train.p', 'wb'))


# pickle.dump(y_train,open('y_train.p', 'wb'))

# pickle.dump(X_val_processed,open('X_val_processed.p', 'wb'))
# pickle.dump(X_test_processed,open('X_test_processed.p', 'wb'))
# pickle.dump(y_val,open('y_val.p', 'wb'))
# pickle.dump(y_test,open('y_test.p', 'wb'))


## Pickle Load

In [15]:
X_train = pickle.load(open('X_train.p', 'rb'))
y_train = pickle.load(open('y_train.p', 'rb'))

X_val= pickle.load(open('X_val_processed.p','rb'))
y_val= pickle.load(open('y_val.p','rb'))

X_test= pickle.load(open('X_test_processed.p','rb'))
y_test= pickle.load(open('y_test.p','rb'))

# Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegressionCV
logRegModel = LogisticRegressionCV(cv =5, Cs=10,scoring = 'precision')
logRegModel.fit(X_train,y_train)

/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:7

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring='precision',
           solver='lbfgs', tol=0.0001, verbose=0)

In [22]:
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

# Run CV with 5 folds (Random Forest)

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [100],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 3],
    'n_estimators': [300,500,700]
}
rf = RandomForestClassifier()
rf_grid= GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=True, n_jobs=-2, scoring ='roc_auc')
rf_grid.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done  28 tasks      | elapsed:  5.0min
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# import sklearn.metrics as metrics

# # Run CV with 5 folds (Random Forest)

# # Create the parameter grid based on the results of random search 
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [300,400,500],
#     'max_features': ['sqrt'],
#     'min_samples_leaf': [1],
#     'min_samples_split': [2],
#     'n_estimators': [300,500,600]
# }

# rf = RandomForestClassifier()
# rf_grid_os= GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=10, n_jobs=-1, scoring ='roc_auc')
# rf_grid_os.fit(X_train_os, y_train_os)

In [20]:
rf_grid.best_score_

0.69052174115390264

In [21]:
rf_grid.best_params_

{'bootstrap': True,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 300}

In [ ]:
logRegModelCoef['abs_coef'] = logRegModelCoef['coef'].abs()
logRegModelCoef.sort_values(by = 'abs_coef', ascending = False)

In [18]:
parameter_grid={
    "loss":["deviance"],
    "learning_rate": [0.6],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 3],
    "max_depth":[40,100,160],
    "max_features":["sqrt"],
    "criterion": ["friedman_mse"],
    "subsample":[ 1.0],
    "n_estimators":[125,300,500]
    }
gb_grid = GridSearchCV(GradientBoostingClassifier(), parameter_grid, cv=5, n_jobs=-1,scoring = 'roc_auc', verbose= True)

gb_grid.fit(X_train, y_train)

gb_grid.best_score_


Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 11.5min finished


0.66593155351874744

In [ ]:
# parameter_grid={
#     "loss":["deviance"],
#     "learning_rate": [0.6],
#     'min_samples_leaf': [1, 2],
#     'min_samples_split': [2, 3],
#     "max_depth":[40,50],
#     "max_features":["sqrt"],
#     "criterion": ["friedman_mse"],
#     "subsample":[ 1.0],
#     "n_estimators":[125]
#     }
# gb_grid_ada = GridSearchCV(GradientBoostingClassifier(), parameter_grid, cv=5, n_jobs=-1,scoring = 'roc_auc', verbose= True)

# gb_grid_ada.fit(X_train_ada, y_train_ada)

# gb_grid_ada.best_score_

In [ ]:
# parameter_grid={
#     "loss":["deviance"],
#     "learning_rate": [0.6],
#     'min_samples_leaf': [1, 2],
#     'min_samples_split': [2, 3],
#     "max_depth":[40,50],
#     "max_features":["sqrt"],
#     "criterion": ["friedman_mse"],
#     "subsample":[ 1.0],
#     "n_estimators":[125]
#     }
# gb_grid_os = GridSearchCV(GradientBoostingClassifier(), parameter_grid, cv=5, n_jobs=-1,scoring = 'roc_auc', verbose= True)

# gb_grid_os.fit(X_train_os, y_train_os)

# gb_grid_os.best_score_

In [ ]:
# gb_grid_os.predict(X_test_processed)

## Pickle the models

In [26]:
gb_grid.best_params_

{'criterion': 'friedman_mse',
 'learning_rate': 0.6,
 'loss': 'deviance',
 'max_depth': 40,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 125,
 'subsample': 1.0}

In [27]:
gb_final = GradientBoostingClassifier(criterion='friedman_mse', learning_rate= .6, loss = 'deviance', 
                                         max_depth=40,min_samples_leaf=1, min_samples_split=3, n_estimators=125,
                                        subsample =1.0)
gb_final.fit(X_train,y_train)

pickle.dump(gb_final,open('gb_final.p','wb'))

In [ ]:
# rf_grid_os.best_params_

In [23]:
rf_final = RandomForestClassifier(bootstrap=True, max_depth=100,max_features='sqrt',min_samples_leaf=2,
                                    min_samples_split=2, n_estimators = 300)

rf_final.fit(X_train,y_train)

pickle.dump(rf_final,open('rf_final.p','wb'))

In [24]:
pickle.dump(logRegModel,open('logRegModel.p','wb'))

In [25]:
logRegModel = pickle.load(open('logRegModel.p','rb'))

In [28]:
rf = pickle.load(open('rf_final.p','rb'))
gb = pickle.load(open('gb_final.p','rb'))

In [29]:
X_train = pickle.load(open('X_train.p','rb'))
y_train = pickle.load(open('y_train.p','rb'))

In [30]:
model_names = ["rf","gb","logRegModel"]


model_vars = [eval(n) for n in model_names]
model_list = list(zip(model_names, model_vars))

In [32]:
voting_classifer_soft= VotingClassifier(estimators=model_list,
                                    voting='soft', #<-- sklearn calls this soft voting
                                    n_jobs=-1)
voting_classifer_soft.fit(X_train, y_train)


VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_wei...dom_state=None, refit=True, scoring='precision',
           solver='lbfgs', tol=0.0001, verbose=0))],
         flatten_transform=None, n_jobs=-1, voting='soft', weights=None)

In [33]:
pickle.dump(voting_classifer_soft, open('voting_classifer_soft.p','wb'))

In [34]:
voting_classifer_hard= VotingClassifier(estimators=model_list,
                                    voting='hard', 
                                    n_jobs=-1)
voting_classifer_hard.fit(X_train, y_train)



VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_wei...dom_state=None, refit=True, scoring='precision',
           solver='lbfgs', tol=0.0001, verbose=0))],
         flatten_transform=None, n_jobs=-1, voting='hard', weights=None)

In [35]:
pickle.dump(voting_classifer_hard, open('voting_classifer_hard.p','wb'))